# Support Vector Machine (SVM)

SVM is a supervised learning algorithm which allows to find a __hyperplane__ (or a set of hyperplanes) in order to classify data in a high (or infinite) dimensional features spaces. 
Intuitively, a good separation is achieved by the hyperplane that has the largest distance to the nearest training data points of any class (so-called _functional margin_), since in general the larger the margin the lower the generalization error of the classifier.
It would be nice if, given a training set, we were able to finh a _decision boundary_ that allows us to make all correct and confident (meaning far from the decision boundary) predictions on the training examples.

The figure below shows the decision bondary for a linearly separable problem, with three samples on the margin boundaries, called __support vectors__

<img src="img/decision_boundary.png" alt="Decision Boundary Visualization" width="500">

The number of __support vectors__ can be much smaller than the size of the training set.

## Functional and geometric margins

Given a training example $(x^{(i)} , y^{(i)} )$, we define the __functional margin__ of $(w, b)$ with respect to the training example as
\begin{equation*}
\hat{\gamma}^{(i)} = y^{(i)} (w^T x^{(i)} + b) \ .
\end{equation*}

Note that:
* if $y^{(i)}=1$, then for the functional margin to be large (i.e., for our prediction to be confident and correct), we need $w^T x^{(i)} + b$ to be a large positive number
* if $y^{(i)}=-1$, then for the functional margin to be large (i.e., for our prediction to be confident and correct), we need $w^T x^{(i)} + b$ to be a large negative number
* if $y^{(i)} (w^T x^{(i)} + b) > 0$, then our prediction on this example is correct.

Hence, a large functional margin represents a confident and a correct prediction. There’s one property of the functional margin that makes it not a very good measure of confidence: it is not invariant under rescaling of the parameters $(w,b)$.

In order to overcome this problem we define the __geometric margin__ of $(w,b)$ with respect to a training example $(x^{(i)} , y^{(i)} )$ as
\begin{equation*}
\gamma^{(i)} = y^{(i)} \left( \frac{w^T}{||w||} x^{(i)} + \frac{b}{||w||} \right) \ .
\end{equation*}

The geometric margin is invariant under rescaling of the parameters and if $||w||=1$ geometric margin equals functional margin.

## The optimal margin classifier

Given a training set, it seems that a natural way to find the best decision boundary is to maximize the geometric margins, since this would reflect a very confident set of predictions on the training set and a good “fit” to the training data. 
Specifically, this will result in a classifier that separates the positive and the negative training examples with a high level of confidence.

Assuming that our dataset is linearly separable, meanig that it is always possible to separate the positive and negative examples using some separating hyperplane, one has to solve the following optimization problem
\begin{align*}
max& _{\gamma, w, b}  \quad \gamma \\
& s.t. \quad y^{(i)} (w^T x^{(i)} + b) \geq 1 \ ,\  i = 1, ..., n \\
& || w || = 1 \ .
\end{align*}

I.e., we want to maximize $\gamma$, subject to each training example having functional margin at least $\gamma$. The $||w|| = 1$ constraint moreover ensures that the
functional margin equals to the geometric margin, so we are also guaranteed
that all the geometric margins are at least $\gamma$. Thus, solving this problem will
result in $(w, b)$ with the largest possible geometric margin with respect to the
training set. The problem here is that the $||w|| = 1$ constraint is non convex. 

One can show that the above maximization problem can be reformulated in the following dual minimization problem
\begin{align*}
min&_{\gamma, w, b}  \quad \frac{1}{2} || w ||^2 \\
& s.t. \quad y^{(i)} (w^T x^{(i)} + b) \geq 1 \ ,\  i = 1, ..., n \ .
\end{align*}

In this form it is an optimization problem with a convex quadratic objective function and only linear constraints. Its solution gives us the __optimal margin classifier__.

The solution of the optimization problem allows to find the optimal value of $w$ as
<a id="optimal_w"></a>
\begin{equation} 
w = \sum_{i=1}^n \alpha_i y^{(i)} x^{(i)} \ , 
\tag{1}
\end{equation}

where the $\alpha_i$ are called _dual coefficients_ (they arise as Langrange multipliers) and are __non zero only for the support vectors__.

Once the optimization problem is solved, and we have found the optimal $w$, we can make a prediction for a given new sample x. We would then calculate $w^T x + b$ and predict $y=1$ if and only if this quantity is bigger than zero. This quantity can be written as

\begin{align} 
w^T x + b & = \Big( \sum_{i=1}^n \alpha_i y^{(i)} x^{(i)} \Big)^T x + b \\
& = \sum_{i=1}^n \alpha_i y^{(i)} x^{(i)^T} x + b \\
& = \sum_{i=1}^n \alpha_i y^{(i)} \langle x^{(i)}, x \rangle + b \\
& = \sum_{i=1}^n \alpha_i y^{(i)} K(x^{(i)}, x) + b \ ,
\end{align} \tag{2}
where $K(\cdot, \cdot)$ is the (linear) _kernel_.

This results shows that if we’ve found the $\alpha_i$’s, in order to make a prediction, we have to
calculate a quantity that depends only on the inner product between $x$ and
the points in the training set $x^{(i)}$. Moreover, the $\alpha_i$’s will all
be zero except for the support vector, and thus, many of the terms in the sum
above will be zero, and we need to find only the inner products between
$x$ and the support vectors (of which there is often only a small number) in
order to make our prediction.

## Regularization and the non-separable case

The derivation of the SVM as presented so far assumed that the data is linearly separable. While mapping data to a high dimensional feature space
via $\phi$ does generally increase the likelihood that the data is separable, we can’t guarantee that it always will be so.

To make the algorithm work for non-linearly separable datasets (as well as be less sensitive to outliers), we reformulate our optimization (using $l_1$ regularization) as follows:

\begin{align*}
min&_{\gamma, w, b}  \quad \frac{1}{2} || w ||^2 + C \sum_{i=1}^n  \xi_i \\
& s.t. \quad y^{(i)} (w^T x^{(i)} + b) \geq 1 - \xi_i \ ,\  i = 1, ..., n \\
& \qquad \  \xi_i \geq 0, i = 1, ..., n \ .
\end{align*}

In this formulation, we allow some samples to be at a distance less than $1$ from their correct functional margin.
If an example has functional margin $1 − \xi_i$ (with $\xi_i$ > 0), we would pay a penalty to the objective function being increased by $C\xi_i$.
The term $C$ controls the strength of this penalty, and as a result, acts as an inverse regularization parameter.

As before, we also have that $w$ can be expressed in terms of the $\alpha_i$’s as
given in equation (1), so that after solving the dual problem, we can continue to use equation (2) to make our predictions.

Note that, somewhat surprisingly, in adding $l_1$ regularization, the only change to the dual problem is that what was originally a constraint that $\alpha_i \geq 0$ has now become $0 \leq \alpha_i \leq C$.

In [ ]:
import numpy as np #library that contains multidimensional array and matrix data structures
import pandas as pd #library useful for data structures and data analysis
import seaborn as sns # data visualization library based on matplotlib
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import datasets

from collections import Counter
import cv2

In [ ]:
# Load the data
df = pd.read_csv('iris.csv')
df.head()

In [ ]:
# separete the features and target

X = df_feature=df.loc[:, ["sepal.length",	"sepal.width", "petal.length","petal.width"]]
Y = df.loc[:, ["variety"]]

In [ ]:
# Split the data to train and test dataset

from sklearn.model_selection import train_test_split # split arrays or matrices into random train and test subsets

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
# test_size, if given in float (should be between 0.0 and 1.0), represents the proportion of the dataset to include in the test split.

In [ ]:
X_train.shape

In [ ]:
# Support vector machine algorithm

from sklearn.svm import SVC
svm = SVC(kernel='linear', C=1, probability=True)
# kernel: specifies the kernel type to be used in the algorithm
# C: regularization parameter. The strength of the regularization is inversely proportional to C.
# probability: whether to enable probability estimates

svm.fit(X_train, y_train)

Let us access the parameters 

In [ ]:
print("kernel: ",svm.kernel) # print the kernel used
print("dual coefficients:\n",svm.dual_coef_) # print the dual coefficients, i.e, the products y_i * alpha_i
print("support vectors:\n",svm.support_vectors_) # print the support vectors
print("number of support vectors:\n",len(svm.support_vectors_)) # print the total number of support vectors
print("number of support vectors for each class:\n",svm.n_support_) # print the number of support vectors for each class
print("number of support vectors:\n",svm.n_support_.sum()) # print the total number of support vectors
print("intercept:\n",svm.intercept_) # print the intercept
print("w: \n",svm.coef_) # print the weight vector

As one can sees, the number of support vectors is $26$ which is much smaller than the number if training example, that is $120$.

In [ ]:
from sklearn.metrics import accuracy_score

# Predict from the test dataset
predictedProbability = svm.predict_proba(X_test)
print("Distribution of probability of a sample to belong to a class\n", predictedProbability) #predicted-probability for each sample to belong to a class

predictions = svm.predict(X_test)
print("vector of top predictions\n",predictions) #label-predicted

print("test set\n",y_test)

# Calculate the accuracy
accuracy_score(y_test, predictions)

# Classification Metrics

## Confusion matrix

A confusion matrix is a way to evaluate the accuracy of a classification.

<img src="img/confusion_matrix.png" alt="Confusion Matrix" width="600">

__Precision__ and __Recall__ have to be defined for each class.

* __Accuracy__: how often the model is correct globally. It is good when classes are balanced. In imbalanced datasets it can be misleading. 
* __Recall__ (Sensitivity/ True Positive Rate (TPR)): out of all actual positive, how many were correctly identified. High Recall = few false negatives.
* __Precision__: of all predicted positives, how many were true. High Precision = few false positives.
* __Specificity__ (True Negative Rate (TNR))= $\frac{TN}{TN + FP}$. High Specificity = few false positives.
* __F1 Score__: $F1 = 2 \frac{Precision \cdot Recall}{Precision + Recall}$. It describes the harmonic mean between precision and recall. It favors balance and is useful when the dataset is imbalanced or both false positive and false negative matter.


In [ ]:
# confusion matrix

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

classes=["Setosa", "Virginica", "Versicolor"]
cm = confusion_matrix(y_test, predictions, labels=classes)
cmd = ConfusionMatrixDisplay(cm, display_labels=classes)
cmd.plot()

### Receiver Operating Characteristic (ROC) Curve & AUC (Area Under Curve)

* The __ROC curve__ is a graphical representation of a classification model's ability to distinguish between positive and negative classes across different threshold values. It plots the __True Positive Rate (TPR or Recall)__ vs. __False Positive Rate__ (FPR = FP/(FP + TN)) at different thresholds.

* The __AUC__ measures a model's ability to distinguish between positive and negative classes. In particulat it gives the probability that the classifier ranks a randomly chosen positive higher than a randomly chosen negative.

Interpretation:
* AUC = 1.0: perfect separation of classes -> Perfect Classifier (TPR=1, FPR=0),, the curve reaches the top-left
* AUC = 0.5: random guessing -> diagonal line
* AUC < 0.5: curve below the diagonal -> bad classifier

In [ ]:
from sklearn.preprocessing import LabelBinarizer # binarize labels in a one-vs-all fashion

label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
y_onehot_test.shape  # (n_samples, n_classes)

# predictedProbability

In [ ]:
print(y_onehot_test)

In [ ]:
from sklearn.metrics import roc_auc_score # compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.

micro_roc_auc_ovr = roc_auc_score(
    y_onehot_test,
    predictedProbability,
    multi_class="ovr",
    average="micro",
)

print(f"Micro-averaged One-vs-Rest ROC AUC score:\n{micro_roc_auc_ovr:.2f}")

In [ ]:
from sklearn.metrics import roc_curve, auc # compute Receiver operating characteristic (ROC)
fpr, tpr, roc_auc = dict(), dict(), dict()

n_classes = 3
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_onehot_test[:, i], predictedProbability[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

fpr_grid = np.linspace(0.0, 1.0, 1000)

# Interpolate all ROC curves at these points
mean_tpr = np.zeros_like(fpr_grid)

for i in range(n_classes):
    mean_tpr += np.interp(fpr_grid, fpr[i], tpr[i])  # linear interpolation

# Average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = fpr_grid
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

print(f"Macro-averaged One-vs-Rest ROC AUC score:\n{roc_auc['macro']:.2f}")

In [ ]:
from itertools import cycle
from sklearn.datasets import load_iris
from sklearn.metrics import RocCurveDisplay

iris = load_iris()
target_names = iris.target_names
fig, ax = plt.subplots(figsize=(6, 6))

'''plt.plot(
    fpr["micro"],
    tpr["micro"],
    label=f"micro-average ROC curve (AUC = {roc_auc['micro']:.2f})",
    color="deeppink",
    linestyle=":",
    linewidth=4,
)

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label=f"macro-average ROC curve (AUC = {roc_auc['macro']:.2f})",
    color="navy",
    linestyle=":",
    linewidth=4,
)'''

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for class_id, color in zip(range(n_classes), colors):
    RocCurveDisplay.from_predictions(
        y_onehot_test[:, class_id],
        predictedProbability[:, class_id],
        name=f"ROC curve for {target_names[class_id]}",
        color=color,
        ax=ax,
    )

plt.plot([0, 1], [0, 1], "k--", label="ROC curve for chance level (AUC = 0.5)")
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Extension of Receiver Operating Characteristic\nto One-vs-Rest multiclass")
plt.legend()
plt.show()